In [1]:
import pandas as pd

# Step 1: Load Dataset
file_path = "churn_analysis.xlsx"
df = pd.read_excel(file_path)

# Quick overview

print("Total rows:", len(df))
print(df.head())

Total rows: 7032
   customerID   TenureGroup  gender  SeniorCitizen Partner Dependents  tenure  \
0  7590-VHVEG   0-12 months  Female              0     Yes         No       1   
1  5575-GNVDE  25-48 months    Male              0      No         No      34   
2  3668-QPYBK   0-12 months    Male              0      No         No       2   
3  7795-CFOCW  25-48 months    Male              0      No         No      45   
4  9237-HQITU   0-12 months  Female              0      No         No       2   

  PhoneService     MultipleLines InternetService  ... TechSupport StreamingTV  \
0           No  No phone service             DSL  ...          No          No   
1          Yes                No             DSL  ...          No          No   
2          Yes                No             DSL  ...          No          No   
3           No  No phone service             DSL  ...         Yes          No   
4          Yes                No     Fiber optic  ...          No          No   

  Streami

In [2]:
# Step 2: Data Cleaning

# Clean 'tenure' column (convert to numeric, handle text/errors)
df['tenure'] = pd.to_numeric(df['tenure'], errors='coerce')

# Clean 'MonthlyCharges' and 'TotalCharges' (numeric)
df['MonthlyCharges'] = pd.to_numeric(df['MonthlyCharges'], errors='coerce')
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Drop rows with missing tenure or charges
df = df.dropna(subset=['tenure', 'MonthlyCharges', 'TotalCharges'])


In [3]:
# Step 3: KPI Summary

total_customers = len(df)
churned_customers = df[df['Churn'] == 'Yes'].shape[0]
churn_rate = churned_customers / total_customers
avg_tenure = df['tenure'].mean()
avg_monthly_charges = df['MonthlyCharges'].mean()

print("\nKPI Summary:")
print(f"Total Customers: {total_customers}")
print(f"Churned Customers: {churned_customers}")
print(f"Churn Rate: {churn_rate:.2%}")
print(f"Average Tenure: {avg_tenure:.2f} months")
print(f"Average Monthly Charges: ${avg_monthly_charges:.2f}")


KPI Summary:
Total Customers: 7032
Churned Customers: 1869
Churn Rate: 26.58%
Average Tenure: 32.42 months
Average Monthly Charges: $64.80


In [4]:
# Step 4: Create Tenure Groups (Cohorts)
def tenure_group(tenure):
    if tenure <= 12:
        return "0-12 months"
    elif tenure <= 24:
        return "13-24 months"
    elif tenure <= 48:
        return "25-48 months"
    else:
        return "49+ months"

df['TenureGroup'] = df['tenure'].apply(tenure_group)


In [5]:
# Step 5: Retention Flag

df['Retained'] = df['Churn'].apply(lambda x: 1 if x == "No" else 0)


In [6]:
# Step 6: Cohort Table (Contract vs TenureGroup)
cohort = pd.pivot_table(
    df,
    index='Contract',
    columns='TenureGroup',
    values='Retained',
    aggfunc='mean'
).round(3) * 100

print("\nCohort Table (% Retention):")
print(cohort)


Cohort Table (% Retention):
TenureGroup     0-12 months  13-24 months  25-48 months  49+ months
Contract                                                           
Month-to-month         48.6          62.3          67.1        74.0
One year               89.4          91.9          89.4        87.1
Two year              100.0         100.0          97.8        96.7


In [7]:
# Step 7: Churn by Segment

# Example: Churn rate by Contract
churn_by_contract = df.groupby('Contract')['Churn'].value_counts(normalize=True).unstack().round(3) * 100
print("\nChurn by Contract (%):")
print(churn_by_contract)

# Example: Churn vs Monthly Charges
monthly_charge_stats = df.groupby('Churn')['MonthlyCharges'].mean().round(2)
print("\nAverage Monthly Charges by Churn:")
print(monthly_charge_stats)



Churn by Contract (%):
Churn             No   Yes
Contract                  
Month-to-month  57.3  42.7
One year        88.7  11.3
Two year        97.2   2.8

Average Monthly Charges by Churn:
Churn
No     61.31
Yes    74.44
Name: MonthlyCharges, dtype: float64


In [8]:
# Step 8: Export to Excel

output_file = "churn_analysis.xlsx"
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name='Cleaned_Data', index=False)
    cohort.to_excel(writer, sheet_name='Cohort_Table')
    
    # KPI Summary
    kpi_df = pd.DataFrame({
        'Metric': ['Total Customers', 'Churned Customers', 'Churn Rate', 'Avg Tenure', 'Avg Monthly Charges'],
        'Value': [total_customers, churned_customers, f"{churn_rate:.2%}", round(avg_tenure,2), round(avg_monthly_charges,2)]
    })
    kpi_df.to_excel(writer, sheet_name='KPI_Summary', index=False)
    
    # Churn by Contract
    churn_by_contract.to_excel(writer, sheet_name='Churn_By_Contract')
    
print("\nExcel file created:", output_file)



Excel file created: churn_analysis.xlsx


In [9]:
# Step 9: Summary Insights

print("\n--- Key Insights ---")
print("1. Month-to-month customers have highest churn.")
print("2. Long-term contracts (1 year, 2 year) have much higher retention.")
print("3. Churned customers pay higher monthly charges on average.")
print("4. Highest churn occurs within first 12 months of tenure.")


--- Key Insights ---
1. Month-to-month customers have highest churn.
2. Long-term contracts (1 year, 2 year) have much higher retention.
3. Churned customers pay higher monthly charges on average.
4. Highest churn occurs within first 12 months of tenure.
